In [1]:
using Revise

In [2]:
import LCPSim
import LearningMPC
using JLD2

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.
INFO: Loading HttpServer methods...


In [3]:
data = jldopen("../2018-03-16-hopper-value-interval-dagger/hopper.jld2")

JLDFile /home/rdeits/locomotion/explorations/learning-mpc-2/experiments/2018-03-16-hopper-value-interval-dagger/hopper.jld2 (read-only)
 ├─🔢 datasets
 ├─🔢 net
 ├─🔢 lqrsol
 ├─🔢 mpc_params
 ├─🔢 all_training_data
 ├─🔢 all_validation_data
 └─🔢 losses

In [5]:
data["lqrsol"].x0

4-element Array{Float64,1}:
 2.0 
 0.75
 0.0 
 0.0 

In [7]:
data["lqrsol"].Q

4×4 Array{Float64,2}:
 10.0  0.0  0.0    0.0
  0.0  0.1  0.0    0.0
  0.0  0.0  0.001  0.0
  0.0  0.0  0.0    1.0

In [8]:
data["lqrsol"].R

2×2 Array{Float64,2}:
 0.001  0.0  
 0.0    0.001

In [6]:
data["lqrsol"].S

4×4 Array{Float64,2}:
 200.0  0.0  0.0    0.0
   0.0  2.0  0.0    0.0
   0.0  0.0  0.02   0.0
   0.0  0.0  0.0   20.0

In [10]:
data["lqrsol"].u0

2-element Array{Float64,1}:
 -0.0 
  9.81

In [2]:
using LearningMPC
using LearningMPC.Models
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using RigidBodyDynamics.PDControl
using MomentumBasedControl
using MeshCatMechanisms
using RigidBodySim
using RigidBodySim.Visualization.MeshCatInterface
import LCPSim
using JLD2
using Blink

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/LearningMPC.ji for module LearningMPC.


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [3]:
robot = BoxAtlas();

In [4]:
mvis = MechanismVisualizer(robot)
open(mvis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 6570`, ProcessRunning), TCPSocket(RawFD(51) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(56) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true))))

In [7]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [8]:
rootframe = root_frame(mechanism(robot))
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism(robot), floor)
wall = HalfSpace3D(Point3D(rootframe, 0., 1.0, 0.), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism(robot), wall)

feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(robot), "l_hand_mount"), :right => findbody(mechanism(robot), "r_hand_mount"));

contact_model = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))

for (side, body) in feet
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end

for (side, body) in hands
    add_contact_point!(body, 
        ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end


In [9]:
momentum_based_controller = MomentumBasedController(mechanism(robot))
contacts = add_mechanism_contacts!(momentum_based_controller)

Dict{RigidBodyDynamics.RigidBody{Float64},Array{MomentumBasedControl.ContactSettings,1}} with 14 entries:
  floor               => MomentumBasedControl.ContactSettings[]
  root                => MomentumBasedControl.ContactSettings[]
  world               => MomentumBasedControl.ContactSettings[]
  l_hand_mount        => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  l_foot_sole_dummy   => MomentumBasedControl.ContactSettings[]
  pelvis              => MomentumBasedControl.ContactSettings[]
  floating_base_dummy => MomentumBasedControl.ContactSettings[]
  r_hand_mount        => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  r_foot_sole         => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  r_hand_mount_dummy  => MomentumBasedControl.ContactSettings[]
  r_foot_sole_dummy   => MomentumBasedControl.ContactSettings[]
  l_foot_sole         => MomentumBasedControl.ContactSettings[MomentumBasedCont…
  l_hand_mount_dummy  => MomentumBasedControl.ContactSetti

In [10]:
μ = 0.8
floor_normal = floor.outward_normal
wall_normal = wall.outward_normal

FreeVector3D in "world": [0.0, -1.0, 0.0]

In [25]:
function lqr_mpc_control(τ::AbstractVector, t::Number, state::MechanismState)
   τ .= lqr_mpc_controller(state)
end

function lqr_basic(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= lqrsol(state)
end

function lqr_feedback_linearized(τ::AbstractVector, t::Number, state::MechanismState)
    u0 = LearningMPC.nominal_input(state, [Point3D(default_frame(body), 0., 0, 0) for body in values(feet)])
    τ .= lqrsol.K * (LearningMPC.qv(state) .- lqrsol.x0) .+ u0
end

lqr_feedback_linearized (generic function with 1 method)

In [13]:
# FIXME: referencing global variables
function lqr_high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(momentum_based_controller)
    MomentumBasedControl.reset!(momentum_based_controller)
    regularize_joint_accels!(momentum_based_controller, 0.005)
    
    for (side, body) in feet
        for contactsettings in contacts[body]
            pt_in_world = transform(state, contactsettings.point, root_frame(state.mechanism))
#             max_normal_force = Inf
            max_normal_force = pt_in_world.v[3] >= 5e-2 ? 0.0 : Inf
            set!(contactsettings, 1e-4, μ, 
                transform(state, floor_normal, default_frame(body)),
                max_normal_force)
        end
    end
    
    momentum_based_controller.terminalstatetask.x .= lqrsol.x0
    momentum_based_controller.terminalstatetask.Q .= lqrsol.S .+ lqrsol.Q
    momentum_based_controller.terminalstatetask.q .= 0
    
    # Low level control
    momentum_based_controller(τ, t, state)
    τ
end

lqr_high_level_control (generic function with 1 method)

In [14]:
function matrix_absolute_value(M::AbstractMatrix)
    fact = eigfact(M)
    S = fact[:vectors]
    M = Diagonal(abs.(fact[:values]))
    S * M * inv(S)
end

function net_high_level_control(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(momentum_based_controller)
    MomentumBasedControl.reset!(momentum_based_controller)
    regularize_joint_accels!(momentum_based_controller, 0.05)
    
    for (side, body) in feet
        for contactsettings in contacts[body]
            pt_in_world = transform(state, contactsettings.point, root_frame(state.mechanism))
#             @show pt_in_world
            max_normal_force = pt_in_world.v[3] >= 5e-2 ? 0.0 : Inf
#             @show max_normal_force
            set!(contactsettings, 1e-4, μ, 
                transform(state, floor_normal, default_frame(body)),
                max_normal_force)
        end
    end
    
    body = hands[:left]
    for contactsettings in contacts[body]
        pt_in_world = transform(state, contactsettings.point, root_frame(state.mechanism))
        max_normal_force = (pt_in_world.v[2] <= (1 - 5e-2)) ? 0.0 : Inf
        set!(contactsettings, 1e-1, μ, transform(state, wall_normal, default_frame(body)))
    end
    
    momentum_based_controller.terminalstatetask.x .= lqrsol.x0
    momentum_based_controller.terminalstatetask.Q .= lqrsol.Q
    
    f = x -> net_cost.net(x)[]
    x = LearningMPC.qv(state)
    q = ForwardDiff.gradient(f, x)
    Q = ForwardDiff.hessian(f, x)
    
    momentum_based_controller.terminalstatetask.Q .+= 0.5 .* matrix_absolute_value(Q)
    momentum_based_controller.terminalstatetask.q .= q
    
#     q0, q = net_cost.tangent_net(vcat(configuration(state), velocity(state)))
#     momentum_based_controller.terminalstatetask.q .= vec(Flux.Tracker.data(q))
    
    # Low level control
    momentum_based_controller(τ, t, state)
    τ
end

net_high_level_control (generic function with 1 method)

In [15]:
effort_bounds = LCPSim.all_effort_bounds(mechanism(robot))

function effort_limiter(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= clamp.(τ, effort_bounds)
    τ
end

position_bounds = LCPSim.all_configuration_bounds(mechanism(robot))

function bounds_enforcer(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: handle q̇ vs v correctly
    for i in 1:num_positions(state)
        kp = 1000
        kd = 0.1 * kp
        if configuration(state)[i] > position_bounds[i].upper
            violation = configuration(state)[i] - position_bounds[i].upper
            τ[i] -= kp * violation
            τ[i] -= kd * velocity(state)[i]
        elseif configuration(state)[i] < position_bounds[i].lower
            violation = position_bounds[i].lower - configuration(state)[i]
            τ[i] += kp * violation 
            τ[i] -= kd * velocity(state)[i]
        end
    end
    τ
end

function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end

    

compose (generic function with 1 method)

In [16]:
using JuMP, Gurobi
using StaticArrays

In [15]:
# function make_qp_controller(lqr::LQRSolution)
#     local solver = GurobiSolver(Gurobi.Env(); OutputFlag=0)
#     local Δt = 0.01
#     local feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
#     local ulimits = LCPSim.all_effort_bounds(mechanism(robot))
    
#     function controller(τ::AbstractVector, t::Number, state::MechanismState)
#         model = Model(solver=solver)
        
#         @variable model u[1:num_velocities(state)]
#         LCPSim.setbounds.(u, ulimits)
#         @variable model qnext[1:num_positions(state)]
#         @variable model vnext[1:num_velocities(state)]
#         xnext = vcat(qnext, vnext)
#         δu = u - lqr.u0
#         δx = xnext .- lqr.x0
#         obj = δx' * lqr.Q * δx + δu' * lqr.R * δu + δx' * lqr.S * δx
#         @objective model Min obj
        
#         world = root_frame(state.mechanism)
        
#         τ_ext = Vector{AffExpr}[]
#         for (side, body) in feet
# #             @show side
#             basis_vectors = normalize.([SVector(0., 1, 1), SVector(0., -1, 1)])
# #             basis_vectors = [SVector(0., 0, 1)]
#             λ = @variable(model, [1:(length(basis_vectors))], basename="λ_$(first(string(side)))", lowerbound=0)
#             basis_forces = [
#                 FreeVector3D(world, λ[i] .* basis_vectors[i])
#                 for i in 1:length(λ)
#             ]
#             force = sum(basis_forces)
#             point = Point3D(default_frame(body), 0., 0, 0)
#             point_in_world = transform(state, point, world)

#             wrench = Wrench(point_in_world, force)
#             J = geometric_jacobian(state, path(state.mechanism, body, root_body(state.mechanism)))
#             push!(τ_ext, torque(J, wrench))
# #             @show side τ_ext[end]
#             if point_in_world.v[3] >= 0.01
#                 @constraint model λ .== 0
#             else
#                 Jp = point_jacobian(state, 
#                     path(state.mechanism, root_body(state.mechanism), body), point_in_world)
#                 @constraint model point_velocity(Jp, vnext).v[3] >= 0
#             end
# #             if point_in_world.v[3] <= 0.01
# #                 Jp = point_jacobian(state, path(state.mechanism, root_body(state.mechanism), body), point_in_world)
# #                 @constraint model point_velocity(Jp, vnext).v[2] <= 0.1
# #                 @constraint model point_velocity(Jp, vnext).v[2] >= -0.1
# #             end
# #             if side == :left
# #                 @constraint model λ .== 0
# #             else
# #                 @constraint model λ .== 20
# #             end
#         end
        
#         bias = dynamics_bias(state)
#         H = mass_matrix(state)
#         HΔv = H * (vnext - velocity(state))
#         @constraint(model, HΔv .== Δt .* (u .- bias .- sum(τ_ext)))
#         q̇next = RigidBodyDynamics.velocity_to_configuration_derivative_jacobian(state) * vnext
#         @constraint(model, qnext .- configuration(state) .== Δt .* q̇next)
        
#         JuMP.solve(model)
# #         @show keys(feet)
# #         @show collect(zip(model.colNames[end-1:end], model.colVal[end-1:end]))
        
        
# #         @show getvalue.(vnext)
#         τ .= getvalue.(u)
#     end
# end

# simple_qp_controller = make_qp_controller(lqrsol)

In [58]:
function make_qp_controller(lqr::LQRSolution)
    local solver = GurobiSolver(Gurobi.Env(); OutputFlag=0)
    local Δt = 0.05
    local feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
    local ulimits = LCPSim.all_effort_bounds(mechanism(robot))
    local xnext = LCPSim.LinearizedState{Variable}(nominal_state(robot))
    
    function controller(τ::AbstractVector, t::Number, state::MechanismState)
        model = Model(solver=solver)
        
        LCPSim.semi_implicit_update!(xnext, state, Δt)
#         LCPSim.set_linearization_configuration!(xnext, configuration(state))
#         LCPSim.set_linearization_velocity!(xnext, velocity(state))
        
        @variable model u[1:num_velocities(state)]
        LCPSim.setbounds.(u, ulimits)
        @variable model qnext[1:num_positions(state)]
        LCPSim.set_current_configuration!(xnext, qnext)
        @variable model vnext[1:num_velocities(state)]
        LCPSim.set_current_velocity!(xnext, vnext)
        
        ū = u - lqr.u0
        x̄ = -(vcat(qnext, vnext) .- lqr.x0)
        obj = x̄' * lqr.Q * x̄ + ū' * lqr.R * ū + x̄' * lqr.S * x̄
        @objective model Min obj
        
        world = root_frame(state.mechanism)
        
        
        τ_ext = Vector{AffExpr}[]
        
        basis_vectors = normalize.([SVector(0., 1, 1), SVector(0., -1, 1)])
#         basis_vectors = [SVector(0., 0, 1)]
        pv = []
        for (side, body) in feet
#             @show side
            contact = @variable(model, basename="c_$(first(string(side)))", category=:Bin)
            λ = @variable(model, [1:(length(basis_vectors))], basename="λ_$(first(string(side)))", lowerbound=0)
            basis_forces = [
                FreeVector3D(world, λ[i] .* basis_vectors[i])
                for i in 1:length(λ)
            ]
            force = sum(basis_forces)
            point = Point3D(default_frame(body), 0., 0, 0)
            point_in_world = transform(LCPSim.linearization_state(xnext), point, world)

            wrench = Wrench(point_in_world, force)
            J = geometric_jacobian(LCPSim.linearization_state(xnext), path(state.mechanism, body, root_body(state.mechanism)))
            push!(τ_ext, torque(J, wrench))
#             @show side τ_ext[end]
            
            # If contact == 1, then the point must be on the ground
            point_linearized = LCPSim.linearized(xnext) do x
                transform(x, point, world)
            end
            # Assumes a max height of 5 meters
            @constraint model point_linearized.v[3] <= 5 * (1 - contact)
            @constraint model point_linearized.v[3] >= 0
            
            # If contact == 0, then there must be no force
            # Assumes a max contact force of 1000 Newtons
            @constraint model λ .<= (1000 * contact)
        end
        
#         bias = LCPSim.linearized(dynamics_bias, xnext) .+ sum(τ_ext)
        bias = dynamics_bias(state) .+ sum(τ_ext)
#         @show -sum(τ_ext)
        H = mass_matrix(LCPSim.linearization_state(xnext))
        HΔv = H * (vnext - velocity(state))
        @constraint(model, HΔv .== Δt .* (u .- bias))
        q̇next = RigidBodyDynamics.velocity_to_configuration_derivative_jacobian(LCPSim.linearization_state(xnext)) * vnext
        @constraint(model, qnext .- configuration(state) .== Δt .* q̇next)
        
#         @constraint model qnext .== [-0.00459581, 0.824581, 0.000397455, 0.0984652, 0.293066, -0.00404757, 0.00128949, 0.7479, 0.8, 0.824537, 0.804643]
#         @constraint model vnext .== [-0.0919161, 0.0916127, 0.0079491, -2.0307, 1.86131, -0.0809513, 0.0257898, 0.958008, 2.0, 0.0907383, -0.307132]
#         @constraint model u .== [0.0, 0.0, 0.0, -2.0, 2.0, 1.71766, -1.96994, 0.142541, 1.54781, 13.3412, -3.29506]
        
        JuMP.solve(model)
#         @show keys(feet)
        n_basis = length(basis_vectors)
#         @show collect(zip(model.colNames[end-(2n_basis-1):end], model.colVal[end-(2n_basis-1):end]))
#         @show getvalue.(pv)
#         @show (dynamics_bias(LCPSim.linearization_state(xnext)))
#         @show getvalue.(LCPSim.linearized(dynamics_bias, xnext))
#         @show getvalue.(-sum(τ_ext))
#         @show getvalue.(u)
#         @show getvalue.(u .- bias)
#         @show getvalue.(HΔv)
#         @show getvalue.(vnext .- velocity(state))
#         δu = getvalue.(u) .- lqr.u0
#         @show getvalue.(u) getvalue.(qnext) getvalue.(vnext)
#         @show model.objVal
        τ .= getvalue.(u)
    end
end

simple_qp_controller = make_qp_controller(lqrsol)

Academic license - for non-commercial use only


(::controller) (generic function with 1 method)

In [54]:
joints(robot.mechanism)

13-element Array{RigidBodyDynamics.Joint{Float64,JT} where JT<:RigidBodyDynamics.JointType{Float64},1}:
 root_to_world                   
 root_to_floor                   
 floating_base                   
 floor_to_wall                   
 floating_base_dummy_to_pelvis   
 pelvis_to_r_hand_mount_rotation 
 pelvis_to_l_hand_mount_rotation 
 pelvis_to_r_foot_sole_rotation  
 pelvis_to_l_foot_sole_rotation  
 pelvis_to_r_hand_mount_extension
 pelvis_to_l_hand_mount_extension
 pelvis_to_r_foot_sole_extension 
 pelvis_to_l_foot_sole_extension 

In [55]:
state = nominal_state(robot)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_r_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_l_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "floating_base"), [0, 0.80, 0])
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [0, 0, 0.5])
set_configuration!(mvis, configuration(state))
τ = similar(velocity(state))
simple_qp_controller(τ, 0.0, state)
τ

11-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
  0.0      
  0.0      
  0.0      
 -0.045562 
  0.211169 
  0.153941 
 -1.38362  
  0.0226297
 -0.554311 
 13.0602   
 -3.03718  

In [56]:
state = nominal_state(robot)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_r_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_l_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "floating_base"), [0, 0.80, 0])
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [0, 0, 0.5])
lqr_mpc_controller(state)

11-element Array{Float64,1}:
  0.0      
  0.0      
  0.0      
 -0.173548 
  0.346468 
 -1.20644  
 -2.92059  
  0.0117664
 -0.563088 
 13.0838   
 -2.97797  

In [59]:
state = nominal_state(robot)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_r_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_l_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "floating_base"), [0, 0.80, 0])
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [-0.5, 0, 0.])

final_time = 5
Δt = 0.01

composed = PeriodicController(similar(velocity(state)), Δt, compose(simple_qp_controller, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer)

dynamics = Dynamics(mechanism(robot), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback, CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = Δt);

  9.072593 seconds (15.51 M allocations: 1005.463 MiB, 18.20% gc time)


In [41]:
RigidBodySim.animate(mvis, state, sol)